In [67]:
file_path = 'API_KEY.txt'

with open(file_path, 'r') as file:
        api_key = file.read()

In [68]:
import os

# get all file names of receipts in New folder -> receipts that need to be extracted. 
folder_path = 'fotos/New'
file_names = os.listdir(folder_path)
print(file_names)

['20241222_172556.jpg', '20241222_172621.jpg', 'gamma.jpg']


In [69]:
import base64
import requests
import json
import re
import pandas as pd
import datetime

# functions

def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')
  

prompt = """Geef informatie over de aankopen op de bon terug. Dit is een voorbeeld van de output, volg het zelfde format:
{"winkel": "Vomar",
    "producten": [
    {
        "beschrijving": "Wokkels Naturel",
        "aantal": 1,
        "totaal prijs": 2.69
    },
    {
        "beschrijving": "Unox Groente Soep",
        "aantal": 3,
        "totaal prijs": 4.8
    }
    ],
    "totaal prijs": 11.03,
    "totaal aantal producten": 4,
    "datum": "20-12-2024",
    "tijd": 13:34,
    "winkel categorie": "Supermarkt"}

Per product, geef het totaal bedrag en geef alleen het totaal bedrag met korting er van af.
"""
  
# prompt chatgpt
def prompt_chatgpt(base64_image):
  headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
  }

  payload = {
    "model": "gpt-4o-mini",
    "messages": [
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": prompt
          },
          {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{base64_image}"
            }
          }
        ]
      }
    ],
    "max_tokens": 300
  }

  response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
  json_response = response.json()
  txt_response = json_response['choices'][0]['message']['content']
  return txt_response


def extract_csv_from_response(response):
    # remove unnecesary character before and after {}
    response = re.search(r'\{.*\}', response, re.DOTALL).group()
    # convert string into json
    data = json.loads(response)

    # convert json into df
    df = pd.DataFrame(data['producten'])
    df['winkel'] = data['winkel']
    df['datum'] = data['datum']
    df['tijd'] = data['tijd']
    df['winkel categorie'] = data['winkel categorie']
    df['prijs per product*'] = df['totaal prijs']/df['aantal']
    df = df[['winkel', 'datum', 'tijd', 'beschrijving', 'prijs per product*', 'aantal', 'totaal prijs', 'winkel categorie']]

    return df 


In [70]:
display(df_products)

,winkel,datum,tijd,beschrijving,prijs per product*,aantal,totaal prijs,winkel categorie
0,Hema,07-11-2024,13:49,String 3 stuks,8.59,1,8.59,Kleding
1,Hema,07-11-2024,13:49,"Ondergoed 4 voor 24,99",24.99,1,24.99,Kleding
0,Kruidvat,19-12-2024,16:36,Kerstkaarten,2.99,2,5.98,Drogisterij


In [75]:

def create_overview_csv(df_products, elapsed_time):
    row = df_products.iloc[0]
    tijd = row['tijd'].replace(":", "-")
    id = f"{row['winkel']}{row['datum']}{tijd}"
    data = {'winkel':[row['winkel']],
            'datum': [row['datum']],
            'tijd': [row['tijd']],
            'totaal prijs': [df_products['totaal prijs'].sum()],
            'winkel categorie': [row['winkel categorie']],
            'id': [id],
            'time_chatgpt_response':elapsed_time,
            'date extracted': datetime.date.today()
            }
    
    df_overview = pd.DataFrame(data)
    return df_overview, id


In [77]:
import shutil
import time

# for each foto of receipt extract information.
for receipt in file_names[1:2]:
    # encode each foto
    image_path = f"{folder_path}/{receipt}"
    base64_image = encode_image(image_path)

    # prompt chatgpt to extract information from receipt
    start_time = time.time()
    response = prompt_chatgpt(base64_image)
    elapsed_time = time.time() - start_time

    df_products = extract_csv_from_response(response)

    df_overview, id = create_overview_csv(df_products, elapsed_time)


    # save and update csv file
    if os.path.exists("extracted_data/records_products.csv"):
        previous_records = pd.read_csv("extracted_data/records_products.csv")
        df_products = pd.concat([previous_records, df_products])
    
    df_products.to_csv("extracted_data/records_products.csv", index=False)


    if os.path.exists("extracted_data/overview.csv"):
        previous_records = pd.read_csv("extracted_data/overview.csv")
        df_overview = pd.concat([previous_records, df_overview])
    
    df_overview.to_csv("extracted_data/overview.csv", index=False)

    shutil.move(image_path, f"fotos/Done/{id}.jpg")


FileNotFoundError: [Errno 2] No such file or directory: 'fotos/New/20241222_172621.jpg'